In [1]:
import json
from functions.experiment import experiment_iteration
from functions.rnaseq_data_generator import rna_seq_generator
%load_ext autoreload
%autoreload 2
import numpy as np
from functions.saving_data import get_empty_auc_dataset

In [9]:
# Prepare to store data into array
STA_range = [10, 20, 50, 100, 200, 500, 1000, 2000, 5000]
data = get_empty_auc_dataset(STA_range, 2)

In [10]:
# Save data
np.savez('auc_results_origin.npz', auc_data=data)
saved_origin = np.load('auc_results_origin.npz', allow_pickle=True)
saved_origin = saved_origin['auc_data'].item()

np.savez('auc_results_expected.npz', auc_data=data)
saved_expected = np.load('auc_results_expected.npz', allow_pickle=True)
saved_expected = saved_expected['auc_data'].item()

np.savez('auc_results_filtered.npz', auc_data=data)
saved_filtered = np.load('auc_results_filtered.npz', allow_pickle=True)
saved_filtered = saved_filtered['auc_data'].item()

In [11]:
def experiment_classification(n_transcripts, n_IS_effect, IS_effect, n_class_effect, class_effect_mean, STA_range):
    n_repeats = 10
    n_genSTA = 7000 # pocet nagenerovanych STA
    n_genCR = 100
    n_genOT = 100
    _, _, _, gen_origin, gen_expected, gen_IS = rna_seq_generator(
        n_transcripts=n_transcripts,
        distribution="Negative binomial",
        n_STA=n_genSTA, n_CR=n_genCR, n_OT=n_genOT,
        n_IS_effect=n_IS_effect, IS_effect=(IS_effect, IS_effect+0.01),  # IS effect
        is_class_effect=n_class_effect > 0, n_class_effect=n_class_effect, class_effect=(class_effect_mean, 0.1)  # class effect, normal distribution
    )
    mean_auc_results_origin, mean_auc_results_expected, mean_auc_results_filtered = \
        experiment_iteration(STA_range, n_repeats,
                             gen_origin, gen_expected, gen_IS,
                             n_genSTA, n_genCR, n_genOT, "Negative binomial")
    return mean_auc_results_origin, mean_auc_results_expected, mean_auc_results_filtered

In [ ]:
# transcripts = [10, 100, 1000, 10000]
transcripts = [1000]
class_effect_n_transcripts = [0, 2, 5, 10]
class_effect = [0.2, 0.3, 0.4]  # mean of normal distribution

IS_effect_n_transcripts = [2, 5, 10]
IS_effect = [0.3, 0.5, 0.9]  # coefficients beta1..beta3

for t in transcripts:
    for n_IS in IS_effect_n_transcripts:
        print('new iteration')
        for i in IS_effect:
            for n_class in class_effect_n_transcripts:
                for c in class_effect:
                    STA_range = [10, 20, 50, 100, 200, 500, 1000, 2000, 5000] #, 10000]

                    mean_o, mean_e, mean_f = experiment_classification(t, n_IS, i, n_class, c, STA_range)
                    saved_origin[t][n_IS][i][n_class][c]['STA_range'] = STA_range
                    saved_origin[t][n_IS][i][n_class][c]['auc'] = mean_o

                    saved_expected[t][n_IS][i][n_class][c]['STA_range'] = STA_range
                    saved_expected[t][n_IS][i][n_class][c]['auc'] = mean_e

                    saved_filtered[t][n_IS][i][n_class][c]['STA_range'] = STA_range
                    saved_filtered[t][n_IS][i][n_class][c]['auc'] = mean_f

new iteration
Exception when  10  STA samples. Error:  Perfect separation detected, results not available
Exception when  10  STA samples. Error:  NaN, inf or invalid value detected in weights, estimation infeasible.
Exception when  10  STA samples. Error:  NaN, inf or invalid value detected in weights, estimation infeasible.
Exception when  10  STA samples. Error:  NaN, inf or invalid value detected in weights, estimation infeasible.
Exception when  10  STA samples. Error:  Perfect separation detected, results not available
Exception when  10  STA samples. Error:  The first guess on the deviance function returned a nan.  This could be a boundary  problem and should be reported.
Exception when  10  STA samples. Error:  Perfect separation detected, results not available
Exception when  10  STA samples. Error:  Perfect separation detected, results not available
Exception when  10  STA samples. Error:  Perfect separation detected, results not available
Exception when  10  STA samples. Err

/Users/karinabalagazova/miniconda3/envs/RPZ/lib/python3.8/site-packages/statsmodels/genmod/families/family.py:1226: RuntimeWarning: divide by zero encountered in true_divide
  endog_mu = self._clean(endog / mu)
/Users/karinabalagazova/miniconda3/envs/RPZ/lib/python3.8/site-packages/statsmodels/genmod/families/family.py:1226: RuntimeWarning: invalid value encountered in true_divide
  endog_mu = self._clean(endog / mu)
/Users/karinabalagazova/miniconda3/envs/RPZ/lib/python3.8/site-packages/statsmodels/genmod/families/links.py:517: RuntimeWarning: overflow encountered in exp
  return np.exp(z)
/Users/karinabalagazova/miniconda3/envs/RPZ/lib/python3.8/site-packages/statsmodels/genmod/families/family.py:1230: RuntimeWarning: divide by zero encountered in log
  resid_dev -= endog_alpha * np.log(endog_alpha / mu_alpha)
/Users/karinabalagazova/miniconda3/envs/RPZ/lib/python3.8/site-packages/statsmodels/genmod/families/family.py:134: RuntimeWarning: invalid value encountered in multiply
  retur

In [20]:
# Save data
np.savez('auc_results_origin5.npz', auc_data=saved_origin)
np.savez('auc_results_expected5.npz', auc_data=saved_expected)
np.savez('auc_results_filtered5.npz', auc_data=saved_filtered)